```{contents}
```

## Structured Output Prompts (LangChain Perspective)

### What Structured Output Prompts Are

**Structured Output Prompts** are prompts designed so that an LLM returns **data in a fixed, machine-readable structure** (JSON, schema, typed object) instead of free-form text.

> The goal is **reliability and determinism**, not creativity.

In LangChain, structured output is enforced using **output parsers** and **model-level constraints**, not just prompt wording.

---

### Why Structured Output Prompts Are Needed

Free-text responses cause:

* Parsing errors
* Ambiguous fields
* Downstream failures
* Hallucinated keys

Structured outputs enable:

* Typed APIs
* Safe automation
* Validation
* Agent-to-agent communication

---

### Structured Output Prompt vs Plain Prompt

| Aspect         | Plain Prompt     | Structured Output Prompt |
| -------------- | ---------------- | ------------------------ |
| Format         | Natural language | Schema-constrained       |
| Parsing        | Manual           | Automatic                |
| Reliability    | Low              | High                     |
| Production use | ❌                | ✅                        |

---

### Core Components of Structured Output in LangChain

### Prompt

Defines **what** to extract or generate.

### Schema

Defines **how** the output must look.

### Parser

Validates and converts output into Python objects.

---

### Basic Structured Output Prompt (Conceptual)

```text
Extract the following fields in JSON:
- category
- priority
- impact
```

This alone is **not sufficient** in production.

---

### Structured Output with Pydantic (Recommended)

### Step 1: Define a Schema



In [1]:
from pydantic import BaseModel, Field

class Ticket(BaseModel):
    category: str = Field(description="Issue category")
    priority: str = Field(description="High, Medium, or Low")
    impact: str = Field(description="User or system impact")




---

### Step 2: Bind Schema to the Model



In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

structured_llm = llm.with_structured_output(Ticket)




LangChain:

* Generates JSON schema
* Enforces output format
* Retries on invalid output

---

### Step 3: Prompt the Model



In [3]:
result = structured_llm.invoke(
    "Email service is down for finance team"
)

print(result)


category='Email Service' priority='High' impact='Finance team unable to communicate effectively, potentially affecting financial operations and reporting.'




Output:

```python
Ticket(category='Infrastructure', priority='High', impact='Finance team cannot send emails')
```

---

### Structured Output with ChatPromptTemplate

#### Combining Prompt + Schema



In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract structured ticket details."),
    ("human", "{input}")
])

chain = prompt | structured_llm

chain.invoke({"input": "VPN access issue for CEO"})


Ticket(category='VPN Access', priority='High', impact='User')



---

### Structured Output Using Output Parsers (Manual Control)

#### PydanticOutputParser



In [5]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Ticket)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Return data in the required format."),
    ("human", "{input}\n\n{format_instructions}")
])

chain = prompt | llm | parser




LangChain injects formatting instructions explicitly.

---

### JSON-Only Structured Output



In [6]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

chain = prompt | llm | parser




Used when:

* Schema is dynamic
* Validation is external

---

### Structured Output in Agents

#### Tool Outputs as Structured Data

Agents rely on structured output for:

* Tool arguments
* Tool results
* Planning steps

LangChain enforces:

* JSON schema for tools
* Typed arguments

---

### Structured Output in RAG

### Citation-Aware Answers

```python
class Answer(BaseModel):
    answer: str
    sources: list[str]
```

Structured output ensures:

* Answers + citations
* No hallucinated fields

---

### Error Handling & Auto-Fixing

#### OutputFixingParser



In [8]:
from langchain_classic.output_parsers.fix import OutputFixingParser

fixing_parser = OutputFixingParser.from_llm(
    llm=llm,
    parser=parser
)




LangChain:

* Detects invalid output
* Re-prompts model to fix format

---

### Common Mistakes

#### Relying only on prompt text

❌ Not reliable

#### Exposing chain-of-thought

❌ Security risk

#### Overly complex schemas

❌ Higher failure rate

#### Missing retries

❌ Fragile pipelines

---

#### Best Practices

* Use `with_structured_output()` whenever possible
* Keep schemas minimal
* Use enums for classifications
* Combine with low temperature
* Validate outputs strictly

---

### When to Use Structured Output Prompts

| Use Case         | Recommended |
| ---------------- | ----------- |
| APIs             | Yes         |
| Agents           | Yes         |
| RAG answers      | Yes         |
| Chat UI          | Optional    |
| Creative writing | No          |

---

### Interview-Ready Summary

> “Structured output prompts in LangChain enforce schema-constrained LLM outputs using parsers and model bindings, enabling reliable, validated, production-ready responses instead of free-form text.”

---

### Rule of Thumb

* **If a machine reads the output → structure it**
* **If a human reads the output → free text is fine**
* **Production → always structured**